In [ ]:
# Alperen Polat 2210356112

In [2]:
import pandas as pd
import numpy as np
import random
import math
from collections import Counter

In [3]:
df = pd.read_csv("16P.csv")

In [4]:
df.shape

(59999, 62)

In [5]:
df.head()

,Response Id,You regularly make new friends.,You spend a lot of your free time exploring various random topics that pique your interest,Seeing other people cry can easily make you feel like you want to cry too,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",You prefer to completely finish one project before starting another.,You are very sentimental.,You like to use organizing tools like schedules and lists.,...,You believe that pondering abstract philosophical questions is a waste of time.,"You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.",You know at first glance how someone is feeling.,You often feel overwhelmed.,You complete things methodically without skipping over any steps.,You are very intrigued by things labeled as controversial.,You would pass along a good opportunity if you thought someone else needed it more.,You struggle with deadlines.,You feel confident that things will work out for you.,Personality
0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,-1,0,0,0,0,0,ENFP
1,1,0,0,-2,-3,-1,2,-2,0,3,...,0,-2,0,2,0,-1,-1,-1,3,ISFP
2,2,0,0,2,0,-1,2,0,0,1,...,0,2,0,2,-1,0,1,2,1,INFJ
3,3,0,-1,3,-1,0,0,-2,0,-2,...,0,0,-1,-1,0,1,0,-2,-1,ISTP
4,4,0,0,-1,0,2,-1,-2,0,1,...,0,1,0,2,0,1,-1,2,-1,ENFJ


In [6]:
df.drop(["Response Id"], 1, inplace=True)

In [7]:
df["Personality"].replace(["ESTJ", "ENTJ", "ESFJ", "ENFJ", "ISTJ", "ISFJ", "INTJ", "INFJ", "ESTP", "ESFP", "ENTP", "ENFP", "ISTP", "ISFP", "INTP", "INFP"],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], inplace=True)

In [13]:
X = df.drop("Personality", axis=1) # This is X column 
X = X.values
Y = df["Personality"] # This Y column 
Y = Y.values

# Euclidean Distance with vector multiplication

In [8]:
def distance(X,X_train):
    num_test = X.shape[0]
    num_train = X_train.shape[0]
    dists = np.zeros((num_test, num_train))
    dists = np.sqrt((X**2).sum(axis=1)[:, np.newaxis] + (X_train**2).sum(axis=1) - 2 * X.dot(X_train.T))
    return dists
    

In [9]:
class KNN():
    def __init__(self, k_number): # I used constructor method 
        self.k = k_number
    def fit(self, training_data, training_label): # This is how I embed the data in the class. to quickly access other functions
        self.X_train = training_data
        self.Y_train = training_label
    def predict(self, test_data):
        predictions = self._predict(test_data) # Normally, I was doing things in a single predict, I saw this way on the Assembly AI channel on YouTube. It is easier to do the operations in a different _predict function within the predict function.
        return predictions
    def _predict(self, X):
        distances = distance(X, self.X_train)
        predictions = []
        neighbors = np.argsort(distances)[:,:self.k]
        for neighbor in neighbors:
            neighbors_labels = [self.Y_train[i] for i in neighbor]
            most_common = Counter(neighbors_labels).most_common()[0][0] #I used the collections library to find the most returning element.
            predictions.append(most_common)     
        
        return predictions

In [10]:
''' I use this function to calculate accuracy , precision and recall '''
def confusion_matrix(predictions, test_label):
    predictions_df = pd.DataFrame(predictions, columns = ['Pred'])
    actual_df = pd.DataFrame(test_label, columns = ['Actual Pred'])
    confusion = pd.crosstab(predictions_df['Pred'], actual_df['Actual Pred'])
    Precision = 0
    Recall = 0
    Accuracy = np.sum(predictions == test_label) / len(test_label)
    for i in range(confusion.shape[0]):
        TP = confusion.iloc[i,i]
        FP = confusion.iloc[i,:].sum() - TP
        FN = confusion.iloc[:,i].sum() - TP
        TN = confusion.sum().sum() - TP-FP-FN
        Precision += TP/(TP+FP)
        Recall += TP/(TP+FN)
    return Accuracy, Precision/16, Recall/16 # I divided by 16 to find the macro average

# k-NN without feature normalization 

In [ ]:
for i in range(1,10,2):
    for j in range(5):
        interval_lower = 12000*(j)
        interval_upper = 12000*(j+1)
        test_data = X[interval_lower:interval_upper, :]
        training_data = np.delete(X, slice(interval_lower,interval_upper), axis=0)
        test_label = Y[interval_lower:interval_upper]
        training_label = np.delete(Y, slice(interval_lower,interval_upper), axis=0)
        print("For k={} croos-validation fold={}".format(i,j+1))
        knn = KNN(i)
        knn.fit(training_data, training_label)
        predictions = knn.predict(test_data)
        accuracy, precision, recall = confusion_matrix(predictions, test_label)
        result_df = pd.DataFrame({"Accuracy":accuracy,
                             "Precision":precision,
                             "Recall": recall}, index=[0])
        print(result_df.head())

    


For k=1 croos-validation fold=1
   Accuracy  Precision    Recall
0  0.978667   0.978665  0.978613
For k=1 croos-validation fold=2
   Accuracy  Precision    Recall
0  0.976833    0.97686  0.976824
For k=1 croos-validation fold=3
   Accuracy  Precision    Recall
0  0.976833   0.976886  0.976863
For k=1 croos-validation fold=4
   Accuracy  Precision    Recall
0  0.978833   0.978882  0.978839
For k=1 croos-validation fold=5
   Accuracy  Precision    Recall
0  0.977581   0.977549  0.977535
For k=3 croos-validation fold=1
   Accuracy  Precision    Recall
0   0.98875   0.988812  0.988712
For k=3 croos-validation fold=2
   Accuracy  Precision    Recall
0   0.98875   0.988717  0.988765
For k=3 croos-validation fold=3
   Accuracy  Precision    Recall
0     0.989    0.98904  0.989041
For k=3 croos-validation fold=4
   Accuracy  Precision    Recall
0  0.987583   0.987606  0.987577
For k=3 croos-validation fold=5
   Accuracy  Precision    Recall
0  0.988249   0.988241  0.988214
For k=5 croos-valida

# k- NN with feature normalization

In [15]:
# min-max normalization
normalized_X_data = (X-np.min(X))/(np.max(X)-np.min(X))

[[0.5        0.5        0.5        ... 0.5        0.5        0.5       ]
 [0.5        0.5        0.16666667 ... 0.33333333 0.33333333 1.        ]
 [0.5        0.5        0.83333333 ... 0.66666667 0.83333333 0.66666667]
 ...
 [0.5        0.5        0.66666667 ... 0.33333333 0.5        0.33333333]
 [0.5        0.5        0.66666667 ... 0.5        0.66666667 0.5       ]
 [0.5        0.5        0.83333333 ... 0.66666667 0.5        0.33333333]]


In [16]:
for i in range(1,10,2): # for k = 1,3,5,7,9
    for j in range(5): # for 5 k-fold cross validation
        interval_lower = 12000*(j)
        interval_upper = 12000*(j+1)
        test_data = normalized_X_data[interval_lower:interval_upper, :]
        training_data = np.delete(normalized_X_data, slice(interval_lower,interval_upper), axis=0)
        test_label = Y[interval_lower:interval_upper]
        training_label = np.delete(Y, slice(interval_lower,interval_upper), axis=0)
        print("For k={} croos-validation fold={} with feature normalization".format(i,j+1))
        knn = KNN(i)
        knn.fit(training_data, training_label)
        predictions = knn.predict(test_data)
        accuracy, precision, recall = confusion_matrix(predictions, test_label)
        result_df = pd.DataFrame({"Accuracy":accuracy,
                             "Precision":precision,
                             "Recall": recall}, index=[0])
        print(result_df.head())

For k=1 croos-validation fold=1 with feature normalization
   Accuracy  Precision   Recall
0  0.978833   0.978836  0.97879
For k=1 croos-validation fold=2 with feature normalization
   Accuracy  Precision   Recall
0   0.97675   0.976757  0.97672
For k=1 croos-validation fold=3 with feature normalization
   Accuracy  Precision    Recall
0  0.977083   0.977143  0.977135
For k=1 croos-validation fold=4 with feature normalization
   Accuracy  Precision    Recall
0  0.978917   0.978962  0.978923
For k=1 croos-validation fold=5 with feature normalization
   Accuracy  Precision    Recall
0  0.978248   0.978218  0.978206
For k=3 croos-validation fold=1 with feature normalization
   Accuracy  Precision    Recall
0  0.988667   0.988731  0.988627
For k=3 croos-validation fold=2 with feature normalization
   Accuracy  Precision    Recall
0  0.988833   0.988802  0.988845
For k=3 croos-validation fold=3 with feature normalization
   Accuracy  Precision    Recall
0  0.989083   0.989127  0.989118
For 

# Error Analysis For Classifacition

- Effect of K_Fold

The purpose of K-fold cross validation is to reduce bias on results.K-Fold did not have much effect in the test. This may be due to the homogeneous distribution of the data, or the K-fold may not have had a significant effect on the data since the data sample consisted of 60000 samples.

- Effect of Normalization

Many machine learning algorithms attempt to find trends in the data by comparing features of data points. However, there is an issue when the features are on drastically different scales.The goal of normalization is to make every datapoint have the same scale so each feature is equally important.But when we use normalization, there is no significant change in the data.Because normalization works when the difference between features is high. For example, if we use KNN for the relationship between the houses sold between 2000-2020 and the values. of the houses, we would have to normalize. Because years are 4 digits, but the value of the house is maybe millions. Our data already varies between -3 and 3. the difference is very little. Min-Max normalization did not cause significant changes in the data. 

-  Effect of Neighbor Number (k)


Lower values of k can have high variance, but low bias, and larger values of k may lead to
high bias and lower variance. When k=1, the accuracy was 97.8%, but when k increased, this rate increased to 98.8% with k=3. When k=5,7,9 there was no significant change.I said that there is no significant change for k=3,5,7,9. But since k=5 is an average neighbor number, I'm basing it on that.

-  Accuracy

Accuracy value averaged 98.8%. I verified the correctness of my result using the scikit library in a different file.Accuracy was too high, I attribute this to the large number of datasets. Therefore, the training data we had to train the model was too much. And so we achieved high accuracy.

- Precision ve Recall

As a result, our precision and recall values ​​were high. This ultimately confirms the reliability of our model.

- Misclassified Samples


I came across misclassified examples in the dataset. This situation occurs when there is no maximum value for a certain k value. However, this issue did not affect accuracy much. I think in such cases we should remove these samples from the dataset.